# Tutorial 4: Large-scale weather regime classification

**Content creators**: Fabian Emmerich

**Content reviewers / testers**: Jannik Jauch

In this tutorial, you will try to classify large-scale weather regimes (LSWRs) in NWP data. Since the data is high-dimensional, you will apply a dimensionality reduction algorithm that allows you to run a classifcation algorithm on the data. Then, you will evaluate and interpret the result of the procedure.

## Exercise 1: Dimensionality reduction of a large grid time series

Our dataset represents a time series covering a large grid over the whole of Europe. To allow running a classification algorithm on these data, we first have to reduce the dimensions of the dataset.

A very well-proven dimensionality reduction algorithm is the Principal Component Analysis (PCA). A PCA is an algorithm that tries to find a orthonormal vector basis of a dataset, where each basis vector accounts for the largest amount of variance in a dataset - in descending order. The basis vectors are called Principal Components (PCs). In descending order means that the first PC accounts for the most variance in the dataset, the second for the second most, and so forth.

A PCA will always give as many PCs as the dataset has dimensions. Choosing only a subset of PCs that account for a sufficiently large amount of variance in the dataset then allows projecting the data into a lower-dimensional subspace of the PC space, that still contains enough information about the variance in the original feature space.

Different authors have shown that PCA can also be applied to time series grid data to find the dominant patterns. Here, we will use it to transform our high-dimensional grid time series into a lower-dimensional time series that allows us to perform a clustering algorithm on it to find similar states of our weather system.

To apply a PCA of time series grid data, though, we have to reshape the data. A PCA can only be performed on a 2-D matrix. For time series, the rows of the matrix have to be the time steps of the time series. Hence, to put the grid of the data into a single row, we have to concatenate each row of the grid, i.e. every data point on a horizontal latitude, to one another to receive a row vector. Then, the PCA can be applied on that data.

Gejerally, we will first use the temperature field of the ECMWF IFS HRES model from 2017 to 2020. Since the LSWRs are defined on a daily time scale, the data were resampled to daily means. However, we first want to get an idea of how the procedure works with a smaller dataset located at `/p/project1/training2223/a6/data/ml_level_137_temperature_daily_mean_jan_2020.nc`, which just contains the data from January 2020. The full dataset is stored at `/p/project1/training2223/a6/data/ml_level_137_temperature_daily_mean_2017_2020.nc`. This is the data of a single level $L = 137$, which is roughly the surface level.

### Tasks

1. Load the smaller dataset from January 2020, transform each time step (grid) into a row vector, and concatenate them to a 2D matrix. For a time series with $T$ time steps covering a grid of size $(M \times N)$, the resulting matrix should be of the form $(T \times MN)$.

   *Hint:* xarray does not allow reshaping of the data. Thus, you will have to transform the underlying data to a `np.array`, which allows for such operations.
1. Now, perform the PCA:
   1. Use [`sklearn.decomposition.PCA`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) and use its `fit()` method to run the PCA on the data.
   1. Plot the scree test using the `PCA.explained_variance_ratio_`.
      You can also plot the cumulative variance ratio to see the total
      amount of variance explained for a given $N_\mathrm{PCs}$.
      
      *Hints:* 
      
      - You can use the `utils.create_scree_test_plot()` method from the [`utils`](./utils.py) package located in the folder of this notebook.
      - If plotting manually, you can simply use `np.cumsum()` for the cumulative variance ratio.
   1. Plot the first two or three PCs (`PCA.components_`). To do so, you will have to reshape it to the original grid. What do the components represent?
      *Hint:* You can use the `utils.restore_original_grid_shape()` method from the [`utils`](./utils.py) package located in the folder of this notebook.
1. Re-run the PCA with $N_\mathrm{PCs}~ = 3$.
   Transform the data (reshaped matrix from Task 1) into PC space (`PCA.transform()`) and plot the result in a 3D plot.
   How can this result be intepreted?
   
   *Hints:* 
   
   - With `matplotlib`, you can create 3D plots very nicely using
     ```python
      fig = plt.figure()
      ax = fig.add_subplot(projection="3d")
      ```
      See [here](https://matplotlib.org/stable/gallery/mplot3d/scatter3d.html).
   - You can use the `utils.create_3d_scatter_plot()` method from the [`utils`](./utils.py) package located in the folder of this notebook.
1. *(Optional)* Animate the plot from Task 3 by highlighting the data points. For example, create a plot for every day in the time series, where the data point of the respective day is colored differently. Then put all plots together into an animation (movie or GIF). Use the magic method `%matplotlib notebook` in the notebook, e.g. at the beginning of the respective cell or execute it beforehand, to enable interactive plotting, which allows you to see the animation.

   *Hints:* 
   
   - [`matplotlib.animation.FuncAnimation`](https://matplotlib.org/stable/api/animation_api.html) allows animation of plots. 
   - You can use the `utils.create_3d_animation_scatter_plot()` method from the [`utils`](./utils.py) package located in the folder of this notebook.

In [ ]:
# Solution to Task 1
%load ./solutions/tutorial-4/task-1-1.py

In [ ]:
# Solution to Task 2
%load ./solutions/tutorial-4/task-1-2.py

In [ ]:
# Solution to Task 3
%load ./solutions/tutorial-4/task-1-3.py

In [ ]:
# Solution to Task 4
%load ./solutions/tutorial-4/task-1-4.py

## Exercise 2: Finding clusters of similar weather patterns

The previous exercise has shown that when using a certain amount of PCs $N_\mathrm{PCs}~$ to transform the data in a low-dimensional PC space, this space represents a $N_\mathrm{PCs}~$-dimensional phase space of our weather system.

So if we choose $N_\mathrm{PCs}~ = 3$, the PC space is the 3D phase space of our system. Ultimately, we have found a 3D representation of our highly dimensional grid data. It contains all states that our dynamical system has resided in during the given time period.

Now that we have a low-dimensional representation of our data, we can use these to perform a clustering. By doing so, we try to find dense regions in the phase space of our system representing re-appearing states of our weather systems, i.e. recurring weather patterns.

For the clustering, there are many different algorithms available, each of which is designed for specific problems. There is a very nice overview of these [in the sklearn documentation](https://scikit-learn.org/stable/modules/clustering.html).

Here, we will first use the most common, supervised algorithm KMeans. One parameter KMeans requires is the number of clusters $N_\mathrm{clusters}~~$ that it is supposed to find. Hence, it is a supervised algorithm. The choice for this parameter is not always simple. If you have a great overview of your data, i.e. your data is low-dimensional, and you can already recognize the clusters by eyeball analysis, this choice may be trivial. In our case, though, we do not necessarily know how many clusters we want to or should find. However, we will first choose $N_\mathrm{clusters}~~ = 4$. Why this is a nice choice for our example dataset will become more clear later.

### Tasks

1. Find a set of clusters using the data in PC space.
   1. Outgoing from a PCA with $N_\mathrm{PCs}~ = 3$, apply [sklearn.cluster.KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) with `n_clusters=4`.
   1. Plot the data, where all data points assigned to a cluster are colored equal.
   1. Plot the time series of the labels. What does each individual cluster represent?
   
      *Hint:* You can use the `utils.create_label_time_series_scatter_plot()` method from the [`utils`](./utils.py) package located in the folder of this notebook.

In [ ]:
# Solution to Task 1
%load ./solutions/tutorial-4/task-2-1.py

## Exercise 3: Using an unsupervised clustering approach

As mentioned previously, KMeans is a supervised clustering algorithm. However, for our problem we do not want to use a supervised approach since we do not know the number of recurring LSWRs in advance. Instead, we want our clustering algorithm to find these by its own.

Another problem of KMeans is that it uses randomised initial cluster centers before running its algorithm. Hence, it is non-deterministic. Different runs of KMeans are very likely to produce slightly different results each time. For this purpose, there exist other types of clustering algorithms that utilise different concepts, e.g. such that are density-based.

One very popular density-based cluserting algorithm is DBSCAN (Density-Based Spatial Clustering of Applications with Noise). Given a dataset, DBSCAN tries to find those data points who have a certain amount of neighbouring data points within a given area (e.g. radius in 2D or sphere in 3D). Such points are called _core points_. Core points that are within the radius of another core point are classified as a cluster. Non-core points, i.e. such that are within the radius of a core point but do not have sufficient neighbouring data points themselves, are also assigned to the respective cluster of that core point. Other points, which are neither core points themselves, nor within the range of a core point, are classified as noise.

A crucial parameter that DBSCAN requires is the radius and the minimum amount of neighbouring points that have to lie within the given radius to classify whether a point is a core point or not. Especially the choice for that radius might be tricky. Another density-based algorithm, which extends DBSCAN but avoids the requirement of the core radius, is HDBSCAN (Hierarchical DBSCAN). 

Simply put, HDBSCAN performs DBSCAN for all possible core radii, where it basically starts from the maximum possible core radius, and then step-by-step lowers that radius. As the radius shrinks, the initial cluster, which contains all data points, gets smaller as boundary points are starting to get classified as noise. Eventually, the large cluster is split into two or more smaller sub-clusters, which themselves get split as the radius shrinks, and so forth. HDBSCAN performs this procedure until all data points are smaller than the minimum number of data points required to form a cluster. Thereby, HDBSCAN builds a hierarchy of clusters that represent the result of the conventional DBSCAN for a finite range of core radii.

Having build the cluster hierarchy, HDBSCAN then selects those clusters that are were most stable throughout the process. However, it is always possible to retrieve the exact result that a DBSCAN would yield for a certain core radius.

A possible downside of HDBSCAN, though, is that it produces outliers, which cannot be assigned to any cluster. The only parameter which HDBSCAN requires is the `min_cluster_size`, which defines the minimum cluster size below which a cluster is classified as noise.

### Tasks

1. Use the result of your previous PCA with $N_\mathrm{PCs}~ = 3$ and cluster the data using HDBSCAN. Compare the results to those from the KMeans.
   1. Apply [`hdbscan.HDBSCAN`](https://hdbscan.readthedocs.io/en/latest/how_hdbscan_works.html) on the transformed data with `min_cluster_size=10`.
   1. Plot the data with each cluster and the outliers colored accordingly.
   1. Plot the label time series and compare to the result of the KMeans.
1. Plot the dendogram for the HDBSCAN clustering (also called _condensed tree_). What does it show?
1. (Optional) Repeat Tasks 1 and 2 with different values for `min_cluster_size`.


In [ ]:
# Solution to Task 1
%load ./solutions/tutorial-4/task-3-1.py

In [ ]:
# Solution to Task 2
%load ./solutions/tutorial-4/task-3-2.py